In [39]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [40]:
path_for_dir = ["../data/train", "../data/valid"]

In [41]:
list_folders = path_for_dir

In [42]:
with_mask = []
list_head_folder = []
list_folder = []
k = 1
for head_folder in  list_folders:
    
    folders = os.listdir(head_folder)
    for folder in tqdm(folders):
        list_head_folder.append(head_folder)
        list_folder.append(folder)
        path_to_folder = os.path.join(head_folder, folder)
        human_pixel_mask = np.load(os.path.join(path_to_folder, 'human_pixel_mask.npy'))
        if np.any(human_pixel_mask == 1):
            with_mask.append(True)
        else:
            with_mask.append(False)
            

  0%|          | 0/20529 [00:00<?, ?it/s]

  0%|          | 0/1856 [00:00<?, ?it/s]

In [43]:
data = pd.DataFrame({'folder': list_folder ,
                     'head_folder': list_head_folder,
                     'with_mask': with_mask
                    })

In [46]:
data.tail()

,folder,head_folder,with_mask
22380,9694875351192015,../data/valid,False
22381,983281586146051313,../data/valid,False
22382,991708144078267819,../data/valid,False
22383,993294049726037916,../data/valid,True
22384,994990958969139413,../data/valid,False


In [47]:
from sklearn.model_selection import StratifiedKFold

def kfold_stratified(df):
    df.head()

    skf = StratifiedKFold(n_splits=6, shuffle=True)
    for fold, (train_index, test_index) in enumerate(skf.split(df, df['with_mask'])):
        df.loc[test_index, 'fold'] = int(fold) + 1

    df['fold'] = df['fold'].astype(int)
    # df.to_parquet('../data_kfold.parquet')
    return df

In [48]:
data_kfold = kfold_stratified(data)

In [49]:
data_kfold.groupby('fold').with_mask.value_counts()

fold  with_mask
1     False        2096
      True         1635
2     False        2096
      True         1635
3     False        2096
      True         1635
4     False        2096
      True         1635
5     False        2095
      True         1636
6     False        2095
      True         1635
Name: count, dtype: int64

In [ ]:
path_to_folder = r"../data/train\995018776372307569"

In [ ]:
img = np.load(os.path.join(path_to_folder, 'img.npy'))
human_pixel_mask = np.load(os.path.join(path_to_folder, 'human_pixel_mask.npy'))
human_individual_mask = np.load(os.path.join(path_to_folder, 'human_individual_mask.npy'))

In [ ]:
plt.figure(figsize=(18, 6))
ax = plt.subplot(1, 4, 1)
ax.imshow(img)
ax.set_title('False color image')

ax = plt.subplot(1, 4, 2)
ax.imshow(np.mean(human_individual_mask, axis=3), interpolation='none')
ax.set_title('Concat individual_mask')

ax = plt.subplot(1, 4, 3)
ax.imshow(img)
ax.imshow(human_pixel_mask, interpolation='none')
ax.set_title('Results pixel mask');

ax = plt.subplot(1, 4, 4)
ax.imshow(img)
ax.imshow(human_pixel_mask, cmap='Reds', alpha=.4, interpolation='none')
ax.set_title('Contrail mask on false color image');

In [50]:
data_kfold.to_csv('../data_6kfold.csv')